In [1]:
import csv
import itertools

l = var('l', domain=RR ) #scaling variable 

def cutoff(v,K):
#
# determine the cutoff matrix H_{0..K} with potential v[0:K-1]
#
    A = matrix(SR,K+1,K+1) #start enumerating from 0
    A = A + matrix.toeplitz([0,1] + [0]*(K-1), [1]+[0]*(K-1))
    for k in range(K):
        A[k,k] = v[k]
    return A
    
def lim_pots(v):
#
# determine all limit potentials of a given periodic potential v
#
    v_rev = deepcopy(v)
    lim = shift_pots(v)
    v_rev.reverse()
    lim_rev = shift_pots(v_rev)
    for pot in lim_rev:
        if pot not in lim:
            lim.append(pot)
    return lim
    
def shift_pots(v):
#
# determine all shifts of a given periodic potential v
#
    shift = []
    for c in range(len(v)):
        if v not in shift:
            shift.append(v)
        v = right_shift(v)
    return shift
    
def right_shift(a):
    return [a[-1] , *a[:-1]]

def hagger_filter(H,val):
#
# filter out eigenvalues of H_+ from cutoff matrix eigenvalues according to Hagger's 
# formula, given a concrete scaling factor val
#
    E_filt = []
    for E in H:
        if abs(real(H[E].subs(l=val))).n() < 1:
            E_filt.append(real(E.subs(l=val)).n())
        E_filt.sort()
    return E_filt
        
def build_ev_dict(v, H={}):
#
# build dictionary H where keys are given via eigenvalues of a cutoff and values are the determinants
#
    K = len(v)
    vl = list(map(lambda vv: l*vv,v))
    A = cutoff(vl,K-2)   
    EV = A.eigenvalues()
    
    for E in EV:  
        vlE = list(map(lambda vv: vv - E,vl))
        H[E] = det(cutoff(vlE,K-1))   
    return H

def export_csv(EV, outname='points.csv'):
#
# deprecated
#
    f = open(outname, 'w')
    writefile = csv.writer(f)
    for E in EV:
        writefile.writerow([len(EV[E]),E,*tuple(EV[E])])
    f.close()
    
def build_lambda_dict(v,l_list):
#
# for a single potential build an eigenvalue dictionary with
# keys given in the list l_list and values given by the corresponding eigenvalues
#
    EV = {}
    for ll in l_list:
        EV[ll] = hagger_filter(build_ev_dict(v), ll)
    return EV

def build_lambda_dict_from_list(v_list,l_list):
#
# same as build_lambda_dict but with lists of potentials
#
    print('Busy...')
    EV = {}
    for ll in l_list:
        H = {}
        for vv in v_list:
            build_ev_dict(vv,H)
        EV[ll] = hagger_filter(H, ll)
    print('Done.')
    return EV

In [2]:
v = [1,1,0,1]

f= open('K4_lambda.csv', 'r')
csv_reader = csv.reader(f) 
l_list = list(map(lambda ll: float(ll), list(csv_reader)[0]))
f.close()

export_csv(build_lambda_dict_from_list(lim_pots(v), l_list), 'K4_points.csv')

Busy...
Done.


In [3]:
v = [0,1,1,0,1]

f= open('K4_lambda.csv', 'r')
csv_reader = csv.reader(f) 
l_list = list(map(lambda ll: float(ll), list(csv_reader)[0]))
f.close()

export_csv(build_lambda_dict_from_list(lim_pots(v), l_list), 'K5_points.csv')

Busy...
Done.
